In [61]:
import pandas as pd
import collections
import os
import time
import sys

In [62]:
#功能函数：对列表进行去重
def uniq(a, b):
    for i in a:
        if i not in b:
            b.append(i)
    return b

#功能函数：消去bigg代谢物id的尾标（_c,_p,_e）
def omit_biggend(Metc):
    tool = []
    line = str(Metc)
    c = line[::-1]
    for i in range(len(c)):
        if c[i] == '_':
            if c[i + 1] == '_':
                l = len(c) - (i + 1) - 1
            else:
                l = len(c) - i - 1
            tool.append(line[0:l])
            break
    return tool

#功能函数：消去简化分类后代谢物的尾标（-1，-2等等）
def omit_classiend(Metc):  # 消去简化后的代谢物的尾标
    tool = []
    line = str(Metc)
    c = line[::-1]
    for i in range(len(c)):
        if c[i] == '-':
            l = len(c) - i - 1
            tool.append(line[0:l])
            break
    return tool

In [63]:
#从替换后的代谢物列表中提取代谢物biggid和简化分类后形式的对应关系和可简化分类的代谢物的biggid的集合
def duiyingdic(metabolite_list):
    global dict_id_name,list_id_name
    dict_id_name = {}
    list_id_name=[]
    for line in metabolite_list:
        if line['CxNy'] not in dict_id_name:
            dict_id_name[line['id']] = line['CxNy']#biggid和简化id的词典形式
            list_id_name.append(line['id'])#有简化id的代谢物的biggid

In [64]:
# 在考虑方程式中带有系数的情况下对方程两侧的代谢物进行系数的合并和化简
def replace2(reaction_eq):  # 化简
    tm = str(reaction_eq)
    splitstr = []
    if ' --> ' in tm:
        splitstr = ' --> '
    elif ' <-- ' in tm:
        splitstr = ' <-- '
    elif ' <=> ' in tm:
        splitstr = ' <=> '
    elif ' ->' in tm:
        splitstr = ' ->'
    else:
        print('需要修正反应方程式的连接符')

    metabolites = tm.split(splitstr)
    biglist = []
    l_list = []
    l_list2 = []
    numlist = []
    num = ''
    n = 1
    if metabolites[0] != '':
        mets = metabolites[0].split(" + ")
        for m in mets:
            if ' ' in m:
                num, rm = m.split(" ")
                m = rm
                result = num + ' ' + m
                l_list.append(result)
                l_list2.append(m)
                num = float(num)
                numlist.append(n * num)
                biglist.append(m)
            else:
                l_list.append(m)
                l_list2.append(m)
                numlist.append(n)
                biglist.append(m)
    r_list = []
    r_list2 = []
    num = []
    p = -1
    if metabolites[1] != '':
        mets = metabolites[1].split(" + ")
        for m in mets:
            if ' ' in m:
                num, rm = m.split(" ")
                m = rm
                result = num + ' ' + m
                r_list.append(result)
                r_list2.append(m)
                num = float(num)
                numlist.append(p * num)
                biglist.append(m)
            else:
                r_list.append(m)
                r_list2.append(m)
                numlist.append(p)
                biglist.append(m)
    ano_biglist = []
    for line in biglist:
        ano_biglist.append(line)
    biglist_slim = []
    uniq(ano_biglist, biglist_slim)
    biglist2 = []
    tool = []
    for line in biglist_slim:
        sum = []
        for b in range(len(biglist)):
            if line == biglist[b]:
                sum.append(numlist[b])
        dc = {}
        c = 0
        for dd in sum:
            c = c + dd
        c = '%.6f' % c
        dc[line] = c
        tool.append(dc)
    final_llist = []
    final_rlist = []
    for line in tool:
        num_meta = line.values()
        meta = line.keys()
        meta = str(meta).replace("dict_keys(['", '').replace("'])", '')
        num_meta = str(num_meta).replace("dict_values(['", '').replace("'])", '')
        if float(num_meta) == 1.0 or float(num_meta) == -1.0:
            if float(num_meta) > 0:
                final_llist.append(meta)
            elif float(num_meta) < 0:
                final_rlist.append(meta)
        else:
            if float(num_meta) > 0:
                l_line = str(float(num_meta)) + ' ' + meta
                final_llist.append(l_line)
            elif float(num_meta) < 0:
                num_meta = -1 * float(num_meta)
                r_line = str(num_meta) + ' ' + meta
                final_rlist.append(r_line)

    if final_llist != '' or final_rlist != '':
        rea = ' + '.join(final_llist) + splitstr + ' + '.join(final_rlist)                                                                                                                                                                                                                                                                                                                    
    tm = str(rea)
    return (tm)

In [65]:
#将提供反应方程式中的所有代谢物转换为其相对应的代谢物简化合并形式，并返回替换之后的反应方程式
# 计算结果分析——替换名字
def replace01(reaction_eq):
    tm = reaction_eq
    duiying = r'D:/TIB-C407/Desktop/代谢碳氮简化/excel/'
    splitstr = []
    if ' --> ' in tm:
        splitstr = ' --> '
    elif ' <-- ' in tm:
        splitstr = ' <-- '
    elif ' <=> ' in tm:
        splitstr = ' <=> '
    elif ' ->' in tm:
        splitstr = ' ->'
    else:
        print('需要修正反应方程式的连接符')

    metabolites = tm.split(splitstr)
    l_list=[]
    num=''
    if metabolites[0] !='':
        mets = metabolites[0].split(" + ")
        for m in mets:
            if ' ' in m:
                num,rm=m.split(" ")
                m=rm
                if m in list_id_name:
                    after_m = dict_id_name[m]
                    result=num +' '+after_m
                    l_list.append(result)
            else:
                if m in list_id_name:
                    after_m = dict_id_name[m]
                    l_list.append(after_m)
    r_list = []
    num=[]
    if metabolites[1] !='':
        mets = metabolites[1].split(" + ")
        for m in mets:
            if ' ' in m:
                num,rm=m.split(" ")
                m=rm
                if m in list_id_name:
                    after_m = dict_id_name[m]
                    result=num +' '+after_m
                    r_list.append(result)
            else:
                if m in list_id_name:
                    after_m = dict_id_name[m]
                    r_list.append(after_m)

    difference1=[item for item in l_list if not item in r_list]
    difference2=[item for item in r_list if not item in l_list]
    difference3=[item for item in r_list if item in l_list]
    difference3_ano=[]
    uniq(difference3, difference3_ano)
    if difference1 == [] and difference2 ==[]:
        rea = ''
    else:
        for i in difference3_ano:
            if i in l_list:
                l_list.remove(i)
            if i in r_list:
                r_list.remove(i)
        rea = ' + '.join(l_list) + splitstr + ' + '.join(r_list)
    tm = str(rea)
    if tm != '':
        tm = replace2(tm)
    return (tm)

In [66]:
#将提供的反应方程式按照'metabolites'的格式要求进行拆分
def metareplace(eq):
    tm = str(eq)
    #print(tm)
    splitstr = []
    if ' --> ' in tm:
        splitstr = ' --> '
    elif ' <-- ' in tm:
        splitstr = ' <-- '
    elif ' <=> ' in tm:
        splitstr = ' <=> '
    elif ' ->' in tm:
        splitstr = ' ->'
    else:
        print('需要修正反应方程式的连接符')

    metabolites = tm.split(splitstr)
    od={}
    od=collections.OrderedDict()
    num = ''
    if metabolites[0] != '':
        mets = metabolites[0].split(" + ")
        for m in mets:
            if '.0 ' in m:
                num, rm = m.split(" ")
                m = rm
                num=-float(num)
                od[m]=num
            else:
                num=-1.0
                od[m] = num

    if metabolites[1] != '':
        mets = metabolites[1].split(" + ")
        for m in mets:
            if '.0 ' in m:
                num, rm = m.split(" ")
                m = rm
                num=float(num)
                od[m]=num
            else:
                num=1.0
                od[m] = num
    return od

In [67]:
#对STEP.1中生成文件中的代谢物和简化形式的对应关系进行分辨+归类，分成只含有无简化形式代谢物的代谢物集合set_meta1和只含有带简化形式代谢物的
#代谢物集合set_meta2，另外将所有含有简化形式的代谢物构建集合cn_set
def findset_meta(excel_file):
    global set_meta1, set_meta2,cn_set
    set_meta1 = []
    set_meta2 = []
    excel1 = pd.ExcelFile(excel_file)
    data1 = excel1.parse('meta_CxNy', index_col=None).T.to_dict().values()
    for line in data1:
        omit_id = omit_biggend(line['id'])#将代谢物的biggid的尾标除去
        if line['CxNy'] == '-':
            set_meta1.append(omit_id)#选取无简化形式的代谢物，将它们的biggid构成集合set_meta1
        else:
            set_meta2.append(omit_id)#选取有简化形式的代谢物，将它们的biggid构成集合set_meta2
    cn_set=[]
    data2 = excel1.parse('CxNy', index_col=None).T.to_dict().values()
    for line in data2:
        cn_set.append(line['CxNy'])#利用有简化形式的代谢物的简化形式构建集合cn_set
    #print('无简化形式的代谢物个数',len(set_meta1))
    #print('有简化形式的代谢物个数', len(set_meta2))
    #print('有简化形式的代谢物的简化形式个数', len(set_meta1))

In [68]:
#从（第一步）简化模型——预处理文件.xlsx’中的'react_CxNy'的sheet中提取反应两侧含有相同代谢物简化形式的反应
#（即最后反应两侧相消后输出反应方程为空的反应）的bigg反应id，并构建反应id的集合set_reac00
def find_cancellationreac(excel_reac):#提取可左右互消的反应，建立集合
    global set_reac00,set_reaid_eq
    set_reac00 = []
    set_reaid_eq=[]
    excel1 = pd.ExcelFile(excel_reac)
    data1 = excel1.parse('react_CxNy', index_col=None).T.to_dict().values()
    set_reaid_eq=data1#将所有反应id，反应简化方程和原反应方程的对应关系转入set_reaid_eq集合中
    for line in data1:
        if line['reaction_CNeq'] == '()':#选取反应两侧含有相同代谢物简化形式的反应的id，构建集合
            set_reac00.append(line['id'])
#按照提供的目标代谢物的简化形式，从‘（第一步）简化模型——预处理文件.xlsx’中提取所有简化形式与目标代谢物简化形式相同的代谢物的biggid
def findset_c6(excel_file, target_meta):#查找目标CN格式的代谢物，建立集合set_c6
    global set_c6
    set_c6 = []
    excelc6 = pd.ExcelFile(excel_file)
    datac6 = excelc6.parse('meta_CxNy', index_col=None).T.to_dict().values()
    for line in datac6:
        omit_id = omit_biggend(line['id'])#将代谢物的biggid尾标去除
        if line['CxNy'] == target_meta:#将与目标简化形式有相同简化形式的代谢物的biggid构建成集合set_c6
            set_c6.append(omit_id)
    #print('目标CN格式的代谢物个数',len(set_c6))

In [69]:
#功能函数：清除指定路径文件夹中的所有文件
def del_file(path):
    for i in os.listdir(path):
        path_file = os.path.join(path, i) #// 取文件绝对路径
        if os.path.isfile(path_file):
            os.remove(path_file)
        else:
            del_file(path_file)

In [70]:
#功能函数：对反应两侧的内容进行拆分，返回拆分后的代谢物的列表
def buildreactset(left_or_right):#拆分反应方程的两侧
    if left_or_right == '':
        return left_or_right
    tool = []
    prelist = left_or_right.split(' + ')
    for item in prelist:
        item = omit_biggend(item)
        if ' ' in str(item):
            number_meta = str(item).split(' ')
            #print('number_meta', number_meta)
            r_meta = number_meta[1].replace("']", '')
            rr_meta = []
            rr_meta.append(r_meta)
            #print('rr_meta,type(rr_meta)', rr_meta, type(rr_meta))
            tool.append(rr_meta)
        else:
            tool.append(item)
    return tool

In [71]:
#将反应进行拆分，并进行判断，目标代谢物是存在于反应的左侧（返回0）或是右侧（返回1），若目标代谢物不存在于反应中则返回空‘ ’
def separate_reac(reaction_eq, meta, rid):#拆分整个反应方程，判断目标代谢物在方程的左侧还是右侧
    tm = reaction_eq
    splitstr = []
    if ' --> ' in tm:
        splitstr = ' --> '
    elif ' <=> ' in tm:
        splitstr = ' <=> '
    else:
        print('需要修正反应方程式的连接符')
    mets = tm.split(splitstr)
    try:
        leftlist = buildreactset(mets[0])
        rightlist = buildreactset(mets[1])
    except:
        print("反应替换出错")
    if meta in leftlist:#对目标代谢物是否存在于指定反应方程中和存在于反应方程的左侧或右侧进行判断
        return 0
    elif meta in rightlist:
        return 1
    else:
        return ' '

In [72]:
#结合separate_reac函数进行判断代谢物在反应哪一侧，之后构建目标代谢物分别在反应左侧和右侧的反应id的集合
def find_reac(excel_file, meta):
    excelc6 = pd.ExcelFile(excel_file)
    datac6 = excelc6.parse('react_CxNy', index_col=None).T.to_dict().values()
    rlist_left = []
    rlist_right = []
    for line in datac6:
        rid = line['id']
        req = line['reaction_eq']
        if separate_reac(req, meta, rid) != ' ':
            if separate_reac(req, meta, rid) == 0:
                rlist_left.append(str(rid))
            elif separate_reac(req, meta, rid) == 1:
                rlist_right.append(str(rid))
    return rlist_left, rlist_right

In [73]:
#利用代谢物在反应左右侧的反应的集合的关系，对代谢物进行归类，返回归为一类的代谢物的集合和合并反应的集合
def try_hebing(c6reac):
    c6tool = []
    for line in c6reac:
        c6tool.append(line)

    hebingc1 = []
    hebingmetaC6_1 = []
    fill_list = []
    delete_list = []
    fill_list.append(c6tool[0])
    delete_list = c6tool

    for item in fill_list:
        meta_A = item['代谢物']
        meta_Aleft = item['代谢物在反应左侧的反应集合']
        meta_Aright = item['代谢物在反应右侧的反应集合']
        try:
            delete_list.remove(item)
            #print(len(delete_list))
            #for line in delete_list:
                #print('delete_list',line)
        except:
            if item not in delete_list:
                continue
            exit()
        if len(delete_list) != 0:
            for next in delete_list:
                abAND = []
                abAND2 = []
                meta_B = next['代谢物']
                meta_Bleft = next['代谢物在反应左侧的反应集合']
                meta_Bright = next['代谢物在反应右侧的反应集合']
                aset = []
                bset = []
                if ', ' in meta_Aleft:
                    aset = meta_Aleft.split(', ')
                else:
                    aset.append(meta_Aleft)

                if ', ' in meta_Bright:
                    bset = meta_Bright.split(', ')
                else:
                    bset.append(meta_Bright)
                for acore in aset:
                    for bcore in bset:
                        if acore == bcore and acore != '':  # and acore != ''
                            #print('acore', acore)
                            #print('item', item['代谢物'])
                            #print('next', next['代谢物'])
                            abAND.append(acore)  # 交集
                aset2 = []
                bset2 = []
                if ', ' in meta_Aright:
                    aset2 = meta_Aright.split(', ')
                else:
                    aset2.append(meta_Aright)
                if ', ' in meta_Bleft:
                    bset2 = meta_Bleft.split(', ')
                else:
                    bset2.append(meta_Bleft)
                for acore in aset2:
                    for bcore in bset2:
                        if acore == bcore and acore != '':  # and acore != ''
                            abAND2.append(acore)  # 交集
                all_AND = []
                all_AND = abAND + abAND2  # (所有)
                if len(all_AND) != 0:
                    fill_list.insert(len(fill_list), next)
                    for cell in all_AND:
                        if cell in set_reac00:
                            hebingc1.append(cell)
                            hebingmetaC6_1.append(next['代谢物'])
                            hebingmetaC6_1.append(item['代谢物'])
                else:
                    hebingc1.append(meta_Aleft)
                    hebingc1.append(meta_Aright)
                    hebingmetaC6_1.append(item['代谢物'])
        else:
            hebingc1.append(meta_Aleft)
            hebingc1.append(meta_Aright)
            hebingmetaC6_1.append(item['代谢物'])
    fill_list2=[]
    fill_list2=uniq(fill_list,fill_list2)
    hebingmetaC6_2=[]
    for i in fill_list2:
        #print('fill_list——', i)
        hebingmetaC6_2.append(i['代谢物'])
    c6tool2 = []
    for line in c6reac:
        c6tool2.append(line)
    delete_list2 = c6tool2
    #print('------------------------------')
    hebingc1_list = []
    hebingmetaC6_1_list = []
    uniq(hebingc1, hebingc1_list)
    uniq(hebingmetaC6_2, hebingmetaC6_1_list)
    return hebingc1_list, hebingmetaC6_1_list

In [74]:
def meta_C6combine(c6reac, target_meta):  # 重点修改的合并算法
    c6tool = []
    decreaselist = []
    for line in c6reac:
        c6tool.append(line)
    result_list = []
    resultreac_list = []
    n = 0
    c6_2 = ['']
    while len(result_list) < len(c6reac):#保证在分类时不会漏掉几个代谢物
        n = n + 1
        if len(c6_2) == 0:
            hebingc1_list = []
            hebingmetaC6_1_list = []
        else:
            hebingc1_list, hebingmetaC6_1_list = try_hebing(c6tool)
            #try_hebing函数可将归为一类的代谢物的集合返回
            for line in hebingmetaC6_1_list:
                dicttool = {}
                dicttool = collections.OrderedDict()
                dicttool['CxNy'] = target_meta + '_{}'.format(n)#按照遍历的代谢物的顺序给他们分一类，二类等等
                dicttool['Metabolite'] = line
                result_list.append(dicttool)
            for line in hebingc1_list:
                for i in line.split(', '):
                    if i != '' and i != 'BIOMASS_Ec_iML1515_WT_75p37M' and i != 'BIOMASS_Ec_iML1515_core_75p37M':
                        reactool = {}
                        reactool = collections.OrderedDict()
                        reactool['CxNy'] = target_meta + '_{}'.format(n)
                        reactool['reaction id'] = i
                        for aa in set_reaid_eq:
                            if reactool['reaction id']==aa['id']:
                                reactool['reaction_CNeq']=aa['reaction_CNeq']
                                reactool['reaction_eq']=aa['reaction_eq']
                        resultreac_list.append(reactool)#对上面分为一类的代谢物简化形式的合并反应进行整理，构建集合
            for piece in hebingmetaC6_1_list:
                decreaselist.append(piece)
            after_decreaselist = []
            uniq(decreaselist, after_decreaselist)
            if len(result_list) != len(c6reac):
                c6tool3 = []
                for line in c6reac:
                    c6tool3.append(line)
                c6_2 = []
                for line in c6tool3:
                    if line['代谢物'] not in after_decreaselist:
                        c6_2.append(line)
                c6tool = c6_2#将以归类的代谢物从同一简化形式代谢物的列表中除去，对剩下的代谢物在进行分类
    resultreac_list2 = []
    uniq(resultreac_list, resultreac_list2)
    return result_list, resultreac_list2


In [75]:
#功能函数：将提供的路径下的文件夹中的所有excel文件都读取后再合并为一个列表进行返回。
def date_integrate(rootdir):
    list = os.listdir(rootdir)  # 列出文件夹下所有的目录与文件
    integrate_list=[]
    for i in range(0, len(list)):
        path = os.path.join(rootdir, list[i])
        if os.path.isfile(path):# 你想对文件的操作
            excelintegrate = pd.ExcelFile(path)
            dataintegrate = excelintegrate.parse('Sheet1', index_col=None).T.to_dict().values()
            for line in dataintegrate:
                integrate_list.append(line)
    return integrate_list

In [76]:
#按照提供的模型中的代谢物列表和代谢物简化分类形式的代谢物biggid的对应关系对代谢物列表进行替换并返回替换好了的代谢物列表
def meta_correspond(data1, data2):
    corresponding_metalist = []
    n = 0
    metaset = []
    for line in data1:
        for meta in data2:
            colimetaid = omit_biggend(line['id'])
            colimetaid = str(colimetaid).replace("['", '').replace("']", '')
            dc = {}
            dc = collections.OrderedDict()
            if colimetaid == meta['Metabolite']:
                n += 1
                dc['number'] = n
                dc['CxNy'] = meta['CxNy']
                dc['id'] = line['id']
                dc['name'] = line['name']
                dc['compartment'] = "c"
                dc['charge'] = "0"
                dc['formula'] = line['formula']
                if meta['CxNy'] in metaset:
                    #print('重复代谢物',meta['CxNy'])
                    dc['annotation'] = "已重复"
                else:
                    dc['annotation'] = "OrderedDict([('bigg.metabolite', '" + line['id'] + "')])"
                if meta['CxNy'] not in metaset:
                    metaset.append(meta['CxNy'])
                corresponding_metalist.append(dc)
    return corresponding_metalist


In [77]:
#功能函数：按照提供的文件列表，输出路径，输出的文件名输出excel文件
def xlsx_export(pw_list, filepath, filename):  # 输出文件
    writer = pd.ExcelWriter(filepath + filename)
    pd.DataFrame(pw_list).to_excel(writer, 'Sheet1', index=False)
    writer.save()

In [78]:
#按照提供的模型中的反应列表和经过替换的代谢物列表，对反应列表中的反应方程和'metabolites'进行代谢物替换
def reac_replace(filepath,metabolite_list,data1515,data_aftclass):
    duiyingdic(metabolite_list)
    after_list=[]
    n=0
    metaset=[]
    for line in data1515:
        n+=1
        alist=[]
        dc={}
        dc=collections.OrderedDict()
        dc['reaction_number'] = n
        dc['id'] = line['id']
        dc['name'] = line['name']
        dc['lower_bound'] = line['lower_bound']
        dc['upper_bound'] = line['upper_bound']
        dc['gene_reaction_rule'] = line['gene_reaction_rule']
        dc['objective_coefficient'] = line['objective_coefficient']
        dc['reaction_eq'] =line['reaction_eq']
        aftereq=replace01(line['reaction_eq'])#对反应方程式进行替换
        dc['reaction_CNeq'] = aftereq
        if aftereq!='' and aftereq!=' <=> ':
            d1=metareplace(aftereq)#将反应方程式按照'metabolites'的格式要求进行拆分
            dc['metabolites'] = d1
            if dc['metabolites'] in metaset:#判断反应方程是否重复，重复的反应将标出
                dc['annotation'] = "已重复"
            else:
                dc['annotation'] = "OrderedDict([('bigg.reaction', '" + line['id'] + "')])"
            if d1 not in metaset:
                metaset.append(d1)
            for k, v in d1.items():
                if v > 0:
                    dc['exchange_reac'] = 1
                    break
        else:
            dc['metabolites'] ='OrderedDict([ ])'
            dc['代谢物原因而反应为空'] = 1
        after_list.append(dc)
    class_reaeq=[]
    #这以下的步骤是利用同简化分类类型代谢物的所有合并反应的列表和上面得到的反应对应关系，将反应全名和反应原方程式和合并反应id相对应
    #并将此对应关系返回class_reaeq列表
    for a in data_aftclass:
        for b in after_list:
            if a['reaction id']==b['id'] and b['reaction_CNeq'] =='':
                dc = {}
                dc = collections.OrderedDict()
                dc['简化代谢物id'] = a['CxNy']
                dc['简化代谢物对应反应id'] = a['reaction id']
                dc['name'] = b['name']
                dc['reaction_eq'] = b['reaction_eq']
                dc['reaction_CNeq'] = b['reaction_CNeq']
                class_reaeq.append(dc)
    return class_reaeq,after_list


In [79]:
#根据替换后的反应列表，从中查找是否有反应方程式重复的反应，并返回重复反应的列表
def repeating_reac(after_reac):
    repeating_reac_list=[]
    for line in after_reac:
        if line['metabolites']!='OrderedDict([ ])':
            repeating_reac_list.append(line['metabolites'])
    uniq_reac=[]
    uniq_reac=uniq(repeating_reac_list, uniq_reac)
    repeating_reac_list2=[]
    for i in uniq_reac:
        num=repeating_reac_list.count(i)
        if num !=1:
            for j in after_reac:
                if i == j['metabolites']:
                    repeating_reac_list2.append(j)
    return repeating_reac_list2


In [80]:
if __name__ == '__main__':
    time_start = time.time()  # 对总程序进行计时
    filepath = r'/media/jupyter/zhang_rzh/redM(MC)/'  # 输入文件的路径
    filename_meta = r'（第一步）简化模型——预处理文件(MC).xlsx'  # 输入文件的文件名（STEP.1生成的文件）
    excel = filepath + filename_meta
    # 确定STEP.1中生成文件的路径
    filename_meta1 = r'metacyc18620new-try.xlsx'
    excel1515 = pd.ExcelFile(filename_meta1)
    data1515_meta = excel1515.parse('metabolites', index_col=None).T.to_dict().values()
    data1515_reac = excel1515.parse('reactions', index_col=None).T.to_dict().values()
    #print(len(data1515_meta))
    #print(len(data1515_reac))
    
    # 从模型的excel文件中提取代谢物和反应的信息
    # 简化代谢物分类
    findset_meta(excel)
    # 对STEP.1中生成文件中的代谢物和反应列表进行整理和合并，保证没有重复的代谢物简化分类形式和反应方程
    del_file(filepath + '分类前')  # 预先清空文件夹
    del_file(filepath + '分类后')
    # 为方便之后的文件的输出，提前将输出文件夹清空
    all_classreact = []
    n=0
    
    for target_meta in cn_set:  # 遍历所有代谢物的简化形式
        print('///////////////////////////////////////////')
        #target_meta='C25N13'
        n+=1
        print(n,target_meta)
        findset_c6(excel, target_meta)  # 按照提供的目标代谢物简化形式，将统一简化形式的代谢物收集构建成集合set_c6
#         print('{}形式的代谢物集合:'.format(target_meta), set_c6[:5])
        c6list = []
        c6list = uniq(set_c6, c6list)  # 对上一步构建的set_c6集合进行去重处理（由于除去了代谢物的biggid尾标）
        #print('c6list代谢物列表', c6list)
        find_cancellationreac(excel)  # 提取反应两侧含有相同代谢物简化形式的反应的id，以构建反应id集合set_reac00和保留反应和简化方程式和
        # 原方程式的集合set_reaid_eq
        c6reac = []
        for meta in c6list:  # 遍历所有和目标代谢物简化形式简化形式相同的代谢物的biggid（去尾标后）
            #print(meta)
            rlist_left, rlist_right = find_reac(excel, meta)
            #print(rlist_left, rlist_right)
            if rlist_left != [] or rlist_right != []:
                dict = collections.OrderedDict()
                dict['CxNy'] = target_meta
                dict['代谢物'] = str(meta).replace("]", '').replace("[", '').replace("'", '').replace("'", '')
                dict['代谢物在反应左侧的反应集合'] = str(rlist_left).replace("]", '').replace("[", '').replace("'", '').replace("'",'')
                dict['代谢物在反应右侧的反应集合'] = str(rlist_right).replace("]", '').replace("[", '').replace("'", '').replace("'",'')
                #print(dict)
                c6reac.append(dict)
        # 根据find_reac函数的结果，将代谢物简化形式，代谢物id，代谢物在反应左侧的反应id集合，代谢物在反应右侧的反应id集合
        xlsx_export(c6reac, filepath + '分类前/', target_meta + '试验.xlsx')
        # 将上面得到的列表输出为exce文件（每一种代谢物的简化形式都各输出一个文件）
        C6classify_list, C6classify_reaclist = meta_C6combine(c6reac, target_meta)
        # 输入目标代谢物简化形式和上面求得到代谢物简化形式、代谢物id、代谢物所在反应两侧的反应id的对应关系的集合，对目标代谢物进行分类

        for line in C6classify_reaclist:
            all_classreact.append(line)
        xlsx_export(C6classify_list, filepath + '分类后/', target_meta + '——碳6分类试验.xlsx')
        # 将同一代谢物简化分类形式下的所有bigg形式的代谢物的对应关系输出为excel恩建（每一类简化形式都输出以此）
        #break
#     sys.exit(0)
    before_list = date_integrate(filepath + '分类前')
    after_list = date_integrate(filepath + '分类后')
    # 整合目标路径文件夹中所有excel文件的内容，并将此内容返回
    # 新旧代谢物替换
    metabolite_list = []
    uniq(meta_correspond(data1515_meta, after_list), metabolite_list)
    # meta_correspond函数对模型的代谢物列表进行替换，替换为简化分类后的形式，然后再对返回的替换后的代谢物列表进行去重
    # 新旧反应替换

    class_reaeq, after_reac = reac_replace(filepath, metabolite_list, data1515_reac, all_classreact)
    # 将反应方程式和'reac_replace'都进行了替换，并同时将合并反应与反应方程进行了对应
    repeating_reaction = repeating_reac(after_reac)
    # 从替换后的反应列表中找出重复的反应方程式
    # 输出文件
    writer = pd.ExcelWriter(filepath + '（第二步）简化模型——分类文件(MC).xlsx')
    pd.DataFrame(after_list).to_excel(writer, '重复的代谢物', index=False)
    # 输出同一简化分类代谢物中的所有bigg代谢物id
    pd.DataFrame(class_reaeq).to_excel(writer, '重复代谢物间的合并反应', index=False)
    # 输出同一简化分类形式的代谢物间的合并反应id、全名和方程式
    pd.DataFrame(repeating_reaction).to_excel(writer, '重复反应', index=False)
    # 输出替换后的反应列表中反应方程式重复的反应的所有信息（id、name、原方程式，简化方程式等等）
    pd.DataFrame(metabolite_list).to_excel(writer, '简化代谢物替换', index=False)
    # 输出全部的（有重复）的替换后的代谢物列表
    pd.DataFrame(after_reac).to_excel(writer, '简化反应替换', index=False)
    # 输出全部的（有重复）的替换后的反应列表
    pd.DataFrame(all_classreact).to_excel(writer, '分类后简化代谢物和反应id的关系', index=False)
    # 输出所有代谢物简化分类形式，和所有代谢物简化形式有参与的反应的id和替换后方程式和原方程式
    pd.DataFrame(before_list).to_excel(writer, '简化代谢物id_参与反应', index=False)
    # 输出代谢物简化形式，代谢物id，代谢物在反应左侧的反应id集合，代谢物在反应右侧的反应id集合的对应关系
    writer.save()
    time_end = time.time()
    print('totally cost', time_end - time_start)
    print('finish!')


///////////////////////////////////////////
1 C5
///////////////////////////////////////////
2 C10N4
///////////////////////////////////////////
3 C5N4
///////////////////////////////////////////
4 C21N2
///////////////////////////////////////////
5 C8
///////////////////////////////////////////
6 C5N1
///////////////////////////////////////////
7 C9N2
///////////////////////////////////////////
8 C6N2
///////////////////////////////////////////
9 C1N2
///////////////////////////////////////////
10 C4N2
///////////////////////////////////////////
11 C17N3
///////////////////////////////////////////
12 C15N2
///////////////////////////////////////////
13 C14N2
///////////////////////////////////////////
14 C54
///////////////////////////////////////////
15 C49
///////////////////////////////////////////
16 C39
///////////////////////////////////////////
17 C19
///////////////////////////////////////////
18 C59
///////////////////////////////////////////
19 C8N1
/////////////////////////

///////////////////////////////////////////
159 C56
///////////////////////////////////////////
160 C55
///////////////////////////////////////////
161 C45
///////////////////////////////////////////
162 C66
///////////////////////////////////////////
163 C71
///////////////////////////////////////////
164 C76
///////////////////////////////////////////
165 C46
///////////////////////////////////////////
166 C45N4
///////////////////////////////////////////
167 C6N4
///////////////////////////////////////////
168 C36N7
///////////////////////////////////////////
169 C23
///////////////////////////////////////////
170 C48N1
///////////////////////////////////////////
171 C42N1
///////////////////////////////////////////
172 C46N1
///////////////////////////////////////////
173 C47N1
///////////////////////////////////////////
174 C45N1
///////////////////////////////////////////
175 C43N1
///////////////////////////////////////////
176 C23N1
///////////////////////////////////////////
1

///////////////////////////////////////////
312 C97N19
///////////////////////////////////////////
313 C26N5
///////////////////////////////////////////
314 C55N1
///////////////////////////////////////////
315 C21N9
///////////////////////////////////////////
316 C18N3
///////////////////////////////////////////
317 C52N3
///////////////////////////////////////////
318 C22N5
///////////////////////////////////////////
319 C18N6
///////////////////////////////////////////
320 C21N3
///////////////////////////////////////////
321 C40N4
///////////////////////////////////////////
322 C112N3
///////////////////////////////////////////
323 C37N3
///////////////////////////////////////////
324 C38N3
///////////////////////////////////////////
325 C68N8
///////////////////////////////////////////
326 C27N3
///////////////////////////////////////////
327 C22N3
///////////////////////////////////////////
328 C26N2
///////////////////////////////////////////
329 C26N4
//////////////////////////

///////////////////////////////////////////
464 C23N3
///////////////////////////////////////////
465 C25N3
///////////////////////////////////////////
466 C26N8
///////////////////////////////////////////
467 C21N4
///////////////////////////////////////////
468 C24N5
///////////////////////////////////////////
469 C28N4
///////////////////////////////////////////
470 C32N8
///////////////////////////////////////////
471 C24N9
///////////////////////////////////////////
472 C23N9
///////////////////////////////////////////
473 C22N9
///////////////////////////////////////////
474 C16N8
///////////////////////////////////////////
475 C53N1
///////////////////////////////////////////
476 C110N2
///////////////////////////////////////////
477 C29N3
///////////////////////////////////////////
478 C52N7
///////////////////////////////////////////
479 C28N5
///////////////////////////////////////////
480 C85N5
///////////////////////////////////////////
481 C30N2
///////////////////////////

///////////////////////////////////////////
616 C98N4
///////////////////////////////////////////
617 C43N3
///////////////////////////////////////////
618 C86N3
///////////////////////////////////////////
619 C31N8
///////////////////////////////////////////
620 C8N6
///////////////////////////////////////////
621 C38N8
///////////////////////////////////////////
622 C43N10
///////////////////////////////////////////
623 C25N13
totally cost 135680.27956223488
finish!
